<a href="https://colab.research.google.com/github/kanhaiyaagrawal01/stepper/blob/master/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [29]:
mov_averages = [0,2,5,10,15,20,25,30]
max_buy = 5
max_keep = 20
base_amount = 10000
min_amount = 4000
max_amount = 24000
n_days = 230
import datetime as dt
import pandas as pd
import numpy as np
import random
class Stock:
    
    def __init__(self, symbol, series=None):
        self.symbol = symbol
        self.history = create_price_series(n_days) if series is None else series
        self.states()
        self.index = mov_averages[-1]
        self.advance()
    
    def set_price(self, price, date):
        self.price = price
        self.date = date
    
    def set_date(self, date):
        self.date = date
        self.price = self.history[date]
        self.index = self.history.index.get_loc(date)
        
    def advance(self):
        self.index += 1
        self.set_date(self.history.index[self.index])
        
    def set_index(self, index):
        self.set_date(self.history.index[index])
    
    def get_state(self):
        return self.states[self.index-mov_averages[-1]-1]
        
    def states(self):
        cumsum = np.array(self.history).cumsum()
        self.states = []
        for i in range(1,len(self.history)-mov_averages[-1]):
            state = [self.history[i+mov_averages[-1]]]
            for j in range(1, len(mov_averages)):
                first = cumsum[i+mov_averages[-1]-1-mov_averages[j]]
                last = cumsum[i+mov_averages[-1]-1]
                state.append((last-first)/mov_averages[j])
            self.states.append(state)
        
#     def moving_average(self, lookback):
#         if self.index < lookback:
#             raise Exception('Lookback window too large!!')
#         else:
#             mean = self.history[self.index-lookback:self.index].mean()
# #             print(mean)
#             return mean
        
    def __str__(self):
        return self.__repr__();

    def __repr__(self):
        return f'Stock (Symbol:{self.symbol}, Price:{self.price}, Date:{self.date})'
        
class Trader():
    
    def __init__(self, stock, cash, date):
        self.stock = stock
        self.position = 0
        self.cash = cash
        self.startCash = cash
        self.startDate = date
    
    def buy(self, x):
        # print("buy", x)
        self.position += x
        self.cash -= x*self.stock.price
        # print(self.position, self.get_state())
        
    def sell(self, x):
        self.position -= x
        self.cash += x*self.stock.price
        
    def close(self):
        self.sell(self.position)
    
    def value(self):
        return (self.position*self.stock.price) + self.cash
    
    def returns(self):
        return (self.value() - self.startCash)/self.startCash
    
    def get_state(self):
        res = np.zeros(len(mov_averages)+2)
        st = np.array(self.stock.get_state())
        for i in range(len(mov_averages)):
          res[i] = st[i]
        res[-2] = self.position
        res[-1] = self.returns()
        return res
        
    def __str__(self):
        return self.__repr__();

    def __repr__(self):
        return f'Trader ( {self.stock}, Position:{self.position}, CashValue:{self.cash}, AccountValue:{self.value()}, Returns:{self.returns()*100}%)'
    
# class Strategy():
#     def __init__(self, trader, lookback=30, stopLoss=0.9, stopGain=1.2):
#         self.trader = trader
#         self.lookback = lookback
#         self.stopLoss = stopLoss
#         self.stopGain = stopGain
        
#     def signal(self):
#         if self.trader.returns()+1>=self.stopGain:
#             return -self.trader.position
#         elif self.trader.returns()+1<=self.stopLoss:
#             return -self.trader.position
#         else:
#             try:
#                 avg = self.trader.stock.moving_average(self.lookback)
#                 if self.trader.stock.price > avg:
#                     return -1
#                 else:
#                     return 1
#             except Exception as e:
# #                 print(e)
#                 return 0
            
#     def run(self, startDate, endDate):
#         self.trader.stock.set_date(startDate)
#         while True:
#             sig = self.signal()
#             self.trader.buy(sig)
# #             print(str(trader))
#             if self.trader.stock.date == endDate:
#                 break
#             self.trader.stock.advance()
            

def create_price_series(length, startdate=dt.datetime.now(), initial_price = 100):
    index = np.array(startdate, dtype=np.datetime64) + pd.to_timedelta(np.arange(length), 'D')
    prices = [initial_price]
    for _ in range(1, length):
        prices.append(prices[-1] + 0.1*initial_price*(random.random()-0.5))
    return pd.Series(prices, index=index)


In [30]:
class TradingEnv(Env):
    def __init__(self):
        self.action_space = Discrete(2*max_buy + 1)
        self.observation_space = Box(low=-max_keep, high=200, dtype=np.float64, shape=(10,))
        # self.observation_space = Box(low=np.array([0]*(len(mov_averages))+[-max_keep,min_amount]), high=np.array([200]*(len(mov_averages))+[max_keep, max_amount]))
        
        stock = Stock('AAPL')
        self.trader = Trader(stock, base_amount, dt.datetime.now())
        self.state = self.trader.get_state()
    
    def step(self, action):
        #state
        self.trader.buy(action-max_buy)
        # print("step1", self.trader.position)
        # print("step2", self.trader.get_state())
        self.trader.stock.advance()
        # print("step3", self.trader.position)
        # print("step4", self.trader.get_state())
        self.state = self.trader.get_state()
        
        #reward
        reward = self.trader.value()-base_amount
        
        #done
        if self.trader.stock.index>=n_days-1:
            done = True
        else:
            done = False
        
        #info
        info = {str(self.trader)}
        
        return self.state, reward, done, info
    
    def render(self):
        pass
    def reset(self):
        stock = Stock('AAPL')
        self.trader = Trader(stock, base_amount, dt.datetime.now())
        self.state = self.trader.get_state()
        return self.state

In [31]:
[0]*(len(mov_averages))+[-max_keep,min_amount]

[0, 0, 0, 0, 0, 0, 0, 0, -20, 4000]

In [33]:
env = TradingEnv()
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    info = {}
    while not done:
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        score = reward
        # print(state, state.shape)
    print('Episode={} Score={} Trader={} state'.format(episode, score, state), state.shape, env.trader.value())

Episode=1 Score=-255.51191377454234 Trader=[ 1.12492518e+02  1.06633856e+02  1.04382014e+02  1.00945656e+02
  9.95671797e+01  1.00292216e+02  1.01621262e+02  1.02504994e+02
  5.00000000e+00 -2.55511914e-02] state (10,) 9744.488086225458
Episode=2 Score=-1754.8031919722198 Trader=[50.19166425 50.09741913 50.6802274  48.90599693 47.06255113 44.12494172
 43.55780436 45.03131203 74.         -0.17548032] state (10,) 8245.19680802778
Episode=3 Score=349.7016783463732 Trader=[1.06948681e+02 1.08660742e+02 1.10784607e+02 1.14070136e+02
 1.14296671e+02 1.13273030e+02 1.11459388e+02 1.10041690e+02
 7.10000000e+01 3.49701678e-02] state (10,) 10349.701678346373
Episode=4 Score=711.2793269181748 Trader=[9.57712929e+01 9.86409165e+01 9.93241898e+01 9.64798303e+01
 9.49384573e+01 9.44039128e+01 9.35914469e+01 9.17967358e+01
 3.80000000e+01 7.11279327e-02] state (10,) 10711.279326918175
Episode=5 Score=948.558949399081 Trader=[88.57716363 85.21378483 87.57732976 87.57544962 85.28596076 83.20484462
 82

In [34]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [35]:
states = env.observation_space.shape
actions = env.action_space.n
states

(10,)

In [52]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=(10,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [53]:
del model
model = build_model(states, actions)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 24)                264       
_________________________________________________________________
dense_22 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_23 (Dense)             (None, 11)                275       
Total params: 1,139
Trainable params: 1,139
Non-trainable params: 0
_________________________________________________________________


In [54]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [55]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [56]:
dqn = build_agent(model, actions) 
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: ignored

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))